# LDA model
## Uncleaned data on one committee

Dependencies:
* numpy
* pandas
* matplotlib
* gensim
* pyLDAvis

In [ ]:
__author__ = 'Eliana Ruby'

In [1]:
import os
import re
from pprint import pprint

import numpy as np
import pandas as pd

#Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)


C:\ProgramData\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
C:\ProgramData\Anaconda3\lib\site-packages\past\types\oldstr.py:33: DeprecationWarning: invalid escape sequence \d
  """
C:\ProgramData\Anaconda3\lib\site-packages\nose\config.py:178: DeprecationWarning: invalid escape sequence \.
  """


In [2]:
# imports of our code
from hebrew_stopwords import hebrew_stopwords
from Parser import tokenize

#### Get data

In [3]:
# name of committee to classify
committee_name = u'המדע והטכנולוגיה'

# this data table is the "meta" table we made.
meta = pd.read_csv('meta.csv')

# Get session ids from the committee I want to classify.
# the committee shows up both with and without 'ועדת' at the beginning, so I checked both.
# (I wanted to use CommitteeID) but it wasn't one-to-one.
session_ids = meta.ID[meta.committee_name.isin([committee_name, u'ועדת {}'.format(committee_name)])]
len(session_ids)

518

In [4]:
# read from our "Data" table (protocol parts with their protocol IDs) in chunks
data = pd.DataFrame()
for chunk in pd.read_csv('data.csv', chunksize=10000):
    # store only the ones from our committee
    data = pd.concat([data,chunk[chunk.ID.isin(session_ids)]])
data.tail()

,Unnamed: 0,header,body,ID
6744331,6744331,ג׳ורדנה קטלר,אנחנו נרשום את ההכנסות שלנו מפרסום במדינת ישרא...,435006.0
6744332,6744332,"היו""ר אורי מקלב","אוקיי. ג'ורדנה, אני חייב לסיים. קודם כל תודה ר...",435006.0
6744333,6744333,ג׳ורדנה קטלר,זה בגלל חברת הכנסת סויד.,435006.0
6744334,6744334,רויטל סויד (המחנה הציוני),נכון.,435006.0
6744335,6744335,"היו""ר אורי מקלב",אני חושב שגם חברת הכנסת סויד חושבת אותו דבר. א...,435006.0


#### Clean data

In [9]:
# filter out trash (empty body text/empty headers)
data = data.fillna('')
f_data = data[data.body.apply(lambda x: len(x) > 0)]
f_data = f_data[f_data.header.apply(lambda x: len(x) > 0)]
len(f_data)

120307

In [ ]:
## group protocol parts into one protocol. The topic model can be run on whole protocols instead 
## of parts, but without preprocessing the results aren't great.
# g_data = f_data.groupby('ID')['body'].apply(lambda x: '\n'.join(x)).reset_index()

In [ ]:
# MORE PREPROCESSING SHOULD GO HERE!
# If we'd managed to use yap to lemmatize and filter by part of speech on the whole dataset, it would go here.

In [10]:
# turn each text into a list of tokens (split into words, filtered by a list of stopwords in 
# hebrew_stopwords.py and Parser.py)
preprocessed_data = [tokenize([s]) for s in f_data.body]

In [11]:
# filter out the word '-' (it's already in stopwords but it didn't work for some reason)
preprocessed_data = [[s for s in l if s != '-'] for l in preprocessed_data]
# filter out extremely short protocol parts (they're pretty noisy and they messed up the model)
preprocessed_data = [t for t in preprocessed_data if len(t) > 10 ]

#### Create model

In [12]:
# This is where the LDA happens :)
# Create Dictionary
id2word = corpora.Dictionary(preprocessed_data)

# Create Corpus
texts = preprocessed_data

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

In [13]:
# Build LDA model
# This can take a while to run. Go grab a beer and maybe a better server.
# gensim doesn't include functionality to print how many iterations it's done, which is annoying.
# You can use logging for it, but I never got around to it...
lda_model = gensim.models.ldamodel.LdaModel(corpus = corpus,       # list of texts as lists of tokens
                                           id2word = id2word,      # dictionary mapping words to ids
                                           num_topics = 30,        # number of topics to look for
                                           random_state = 20,      
                                           passes = 10,            # number of iterations over the data (more is better for the model, to a point)
                                           alpha = 'auto',
                                           per_word_topics = True)

In [27]:
# Save model to disk.
temp_file = "models/model6"
lda_model.save(temp_file)
temp_file

C:\ProgramData\Anaconda3\lib\site-packages\smart_open\smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
C:\ProgramData\Anaconda3\lib\site-packages\smart_open\smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
C:\ProgramData\Anaconda3\lib\site-packages\smart_open\smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-fu

'models/model6'

In [21]:
# keep training the model for more iterations
lda_model.update(corpus, passes=10)

In [23]:
# Print the topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(7,
  '0.026*"נשים" + 0.015*"בירושלים" + 0.014*"פרס" + 0.012*"במגזר" + '
  '0.009*"החרדי" + 0.009*"הערבי" + 0.009*"מוזיאון" + 0.008*"מורה" + '
  '0.008*"צעירים" + 0.008*"מחשוב"'),
 (24,
  '0.037*"אדם" + 0.024*"מאגר" + 0.022*"ניסויים" + 0.021*"המאגר" + '
  '0.018*"רפואיים" + 0.017*"בבני" + 0.015*"הלסינקי" + 0.014*"העליונה" + '
  '0.012*"הביומטרי" + 0.010*"הגנטי"'),
 (2,
  '0.033*"הישיבה" + 0.024*"בשעה" + 0.015*"ננעלה" + 0.014*"תודה" + 0.013*"רבה" '
  '+ 0.009*"המחירים" + 0.008*"מודה" + 0.007*"מיקי" + 0.006*"פיילוט" + '
  '0.006*"להצעת"'),
 (21,
  '0.019*"המשפטי" + 0.013*"היועץ" + 0.011*"גיל" + 0.011*"אינטל" + '
  '0.011*"לפגוע" + 0.010*"קופת" + 0.009*"במחשב" + 0.009*"16" + 0.008*"מתייחס" '
  '+ 0.007*"הפגיעה"'),
 (11,
  '0.026*"אביב" + 0.020*"המקומיות" + 0.019*"הרשויות" + 0.017*"רשויות" + '
  '0.017*"המקומי" + 0.015*"תל" + 0.013*"השלטון" + 0.008*"בתל" + 0.008*"תעודות" '
  '+ 0.008*"לימודי"'),
 (1,
  '0.013*"לבית" + 0.013*"חופש" + 0.012*"זמנים" + 0.011*"המורים" + '
  '0.010*"שדיברנו" + 

In [22]:
# calculate topic coherence - this is a measure of how much the topics make sense. 
# The accepted standard for a model is for this to be above 0.6, right now I'm at 0.47...

coherence_model_ldamallet = CoherenceModel(model=lda_model, texts=preprocessed_data, dictionary=id2word, coherence='c_v')
coherence_ldamallet = coherence_model_ldamallet.get_coherence()
coherence_ldamallet

0.47272110136549406

#### Visualization

In [24]:
# Visualize
# this can take a while to run, but it's really cool so you should do it anyway. 
# Also, you can save the visualizations as an html document.
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

C:\ProgramData\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.261422 -0.150102       1        1  27.740578
18    -0.222267 -0.011832       2        1   8.044375
5     -0.271243 -0.012215       3        1   7.929348
28    -0.211567 -0.144006       4        1   5.161180
17    -0.136752  0.134884       5        1   4.895930
23    -0.120057 -0.007428       6        1   4.409428
25    -0.070770  0.055612       7        1   3.423962
20    -0.042360 -0.096811       8        1   3.390276
19    -0.063995  0.272700       9        1   3.288218
27    -0.046711  0.303310      10        1   2.914349
14    -0.050158  0.029906      11        1   2.786125
8      0.031338 -0.011451      12        1   2.503370
12     0.039834 -0.024810      13        1   2.445965
22     0.033211 -0.067623      14        1   1.904202
9      0.056280 -0.012875      15        1   1.818552
15     0.071313 -0.044184      16        1   1.787359
13     0.036003 -0.090378      17        1   1.501791
26     0.072541  0.085534      18        1   1.457531
4      0.060947 -0.046035      19        1   1.431437
6      0.093039  0.007422      20        1   1.295161
10     0.072108 -0.070605      21        1   1.142004
29     0.106110 -0.027814      22        1   1.127852
3      0.087131  0.002435      23        1   1.125896
7      0.085005  0.061453      24        1   1.008620
11     0.098805  0.013960      25        1   1.006222
16     0.105244 -0.044957      26        1   0.980008
1      0.117975 -0.030988      27        1   0.931513
21     0.119203 -0.039366      28        1   0.866035
24     0.116001 -0.047612      29        1   0.843228
2      0.095216  0.013874      30        1   0.839487, topic_info=      Category          Freq        Term         Total  loglift  logprob
96     Default   9097.000000       ישראל   9097.000000  30.0000  30.0000
18     Default  11381.000000        משרד  11381.000000  29.0000  29.0000
453    Default   6450.000000       הכנסת   6450.000000  28.0000  28.0000
3063   Default   5313.000000      הממשלה   5313.000000  27.0000  27.0000
130    Default   6008.000000        המדע   6008.000000  26.0000  26.0000
524    Default  12943.000000        צריך  12943.000000  25.0000  25.0000
58     Default   5257.000000        מידע   5257.000000  24.0000  24.0000
445    Default   4771.000000        שנים   4771.000000  23.0000  23.0000
1074   Default  12212.000000         שיש  12212.000000  22.0000  22.0000
472    Default   3972.000000         אדם   3972.000000  21.0000  21.0000
1479   Default   4938.000000        תודה   4938.000000  20.0000  20.0000
436    Default  12169.000000        רוצה  12169.000000  19.0000  19.0000
525    Default   5869.000000         ראש   5869.000000  18.0000  18.0000
2      Default  13671.000000        היום  13671.000000  17.0000  17.0000
542    Default  10836.000000         אחד  10836.000000  16.0000  16.0000
409    Default   9991.000000       לעשות   9991.000000  15.0000  15.0000
645    Default   3757.000000         שנה   3757.000000  14.0000  14.0000
932    Default   9980.000000         שזה   9980.000000  13.0000  13.0000
901    Default   9713.000000         דבר   9713.000000  12.0000  12.0000
1009   Default   4874.000000        לגבי   4874.000000  11.0000  11.0000
1595   Default   3391.000000       המידע   3391.000000  10.0000  10.0000
281    Default   4171.000000      הוועדה   4171.000000   9.0000   9.0000
491    Default   5841.000000       הנושא   5841.000000   8.0000   8.0000
704    Default  11215.000000        הרבה  11215.000000   7.0000   7.0000
585    Default   8193.000000        ואני   8193.000000   6.0000   6.0000
782    Default  13126.000000      שאנחנו  13126.000000   5.0000   5.0000
2197   Default   4799.000000      בעניין   4799.000000   4.0000   4.0000
178    Default   3283.000000      החינוך   3283.000000   3.0000   3.0000
279    Default   2711.000000     הבריאות   2711.000000   2.0000   2.0000
508    Default   3455.000000        מחק

In [26]:
pyLDAvis.save_html(vis,'models/this_one_works.html')